In [1]:
import pandas as pd
import torch
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import os
import PIL.Image as Image
from torchvision import transforms, utils
import numpy as np
%matplotlib inline

In [2]:
device=torch.device("cuda:0")
net=models.resnet34(pretrained=False , num_classes=2)
net=net.to(device)

In [15]:

class CovidDataset(Dataset):
    """Covid Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with traindata.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
         
        df= pd.read_csv(csv_file)
        self.imList= df['X_ray_image_name'].tolist()
        self.labels= df['Label'].tolist()
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['Normal', 'Pnemonia']

    def __len__(self):
        return len(self.imList)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.imList[idx])
        image = Image.open(img_name)
        image = image.resize((256,256), Image.ANTIALIAS)
        labelStr=self.labels[idx]
        label=np.array(int(labelStr=='Normal'))
        sample = {'image': image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample
    
    
    
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        image =  np.asarray(image)
        if len(image.shape)<3:
            image=np.stack([image,image,image], axis=0)
        else:
            image=np.transpose(image,(2,0,1))
            
        if image.shape[0]!=3:
            print(image.shape)
            image=image[0:3,:,:]
        return {'image': torch.from_numpy(image).type(torch.FloatTensor).to(device),
                'label': torch.from_numpy(label).type(torch.LongTensor).to(device)}


In [16]:
trainPath='../Dataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/'
trainCsv='train.csv'
datasettrain=CovidDataset(trainCsv,trainPath,transform=transforms.Compose([
                                               ToTensor()]))

In [17]:
for i in range(len(datasettrain)):
    sample = datasettrain[i]
    if len(sample['image']>2):
        print(sample['image'].shape)
    

torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([

KeyboardInterrupt: 

In [18]:
dataloader = DataLoader(datasettrain, batch_size=4,shuffle=True)

In [19]:
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data['image'], data['label']

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

(4, 256, 256)
(4, 256, 256)
(4, 256, 256)
(4, 256, 256)
(4, 256, 256)
(4, 256, 256)
(4, 256, 256)
(4, 256, 256)
(4, 256, 256)
(4, 256, 256)
(4, 256, 256)
(4, 256, 256)
(4, 256, 256)
(4, 256, 256)
(4, 256, 256)
(4, 256, 256)
Finished Training


In [106]:
image=sample['image']
image.shape

torch.Size([256, 256])

In [107]:
len(image.shape)

2

In [113]:
type(data['image'])

torch.Tensor